공공데이터포털에 공개된 국민건강보험가입자 진료내역 정보(https://www.data.go.kr/dataset/15007115/fileData.do) 중 2016년 진료내역에 대한 분석을 실시.

본래 1200만여개의 진료기록에 대한 19칼럼의 데이터로 구성되었으나, 100만명의 국민건강보험 가입자들에 대한 90칼럼의 데이터로 재구성해 분석함.

재구성한 데이터는 다음과 같음. 
1. 개인별 정보 : ID/성별/연령대/1년간 총 요양일 수/1년간 총 처방일수/1년간 총 방문횟수 
2. 각 진료별 상병(질병유형)코드 : 주상병 (21칼럼 할당), 부상병(23칼럼 할당)
3. 서식코드 : 의과 입원, 의과 외래, 보건기관 외래에 대해 총 3칼럼 할당
4. 진료과목 코드 : 일반의, 내과, 신경과, 정신과, 외과 등 총 34칼럼 할당
5. 심결가산율 : 0.15(의원, 보건의료원 등), 0.2(일반 병원), 0.3(상급 종합병원)의 3칼럼 할당

기본 데이터에 포함되어있는 '심결본인부담금'을 실제 부담 의료비 금액으로 설정한 후, 개인별 부담금의 8분위 분위수를 계산하였음.

이후 Keras 및 Tensorflow를 이용하여 개인별 데이터를 input으로 받고, 1부터 8까지의 의료비 지출 분위수를 예측 output으로 산출하는 신경망 모델을 구성하였음.


In [0]:
#Load modules and check file configuration
from google.colab import drive
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
import os
import tensorflow
import numpy as np
import pandas as pd
drive.mount("/content/drive/", force_remount = True)
path = '/content/drive/My Drive/ML/건강보험'
os.chdir(path)
data_in_path = './input/'
data_out_path = './out/'

#Load data
data = pd.read_csv(data_in_path + 'gun_train.csv', encoding = 'cp949')
test_data = pd.read_csv(data_in_path + 'gun_test.csv', encoding='cp949')

In [0]:
#Training data generation
data_train = data.loc[:,'성별코드':'quartile']
Response = data_train.pop('quartile')
Response = Response-1

#Into Tf.dataset 
dataset = tensorflow.data.Dataset.from_tensor_slices((data_train.values,Response.values))

#Validation Split
validation_dataset = dataset.take(100000) 
train_dataset = dataset.skip(100000)
train_dataset = train_dataset.shuffle(700000).batch(16)
validation_dataset = validation_dataset.shuffle(100000).batch(16)

In [0]:
#Test data generation
data_test = test_data.loc[:,'성별코드':'quartile']
test_Response = data_test.pop('quartile')
test_Response = test_Response-1

#Into Tf.dataset
dataset_test = tensorflow.data.Dataset.from_tensor_slices((data_test.values,test_Response.values))
test_dataset = dataset_test.shuffle(200000).batch(16)

In [0]:
#Model generation
model = tensorflow.keras.models.Sequential([
  tensorflow.keras.layers.Dense(100, input_shape = (89, ), activation ='relu'),
  tensorflow.keras.layers.Dense(100,activation = 'relu'),
  tensorflow.keras.layers.Dense(100,activation = 'relu'),
  tensorflow.keras.layers.Dense(100,activation = 'relu'),
  tensorflow.keras.layers.Dense(100, activation ='relu'),
  tensorflow.keras.layers.Dense(100,activation = 'relu'),
  tensorflow.keras.layers.Dense(8, activation ='softmax') #최종 노드 값은 합이 1이 되도록 분할한 값 10개.
])

#Set optimizer and compile model
Adam = tensorflow.keras.optimizers.Adam # 최적화 기재를 더 잘 만지고 싶다면, 이런 식으로 객체에 받아온 후에 세부조정 해줘야 함.
model.compile(optimizer= Adam(lr = 0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Check Summary
model.summary()

In [0]:
#If needed, load previous weights
weight_in_path ='./weights/'
model.load_weights(weight_in_path + 'Health_Full_600.h5')

In [0]:
#Train model
from keras.callbacks import CSVLogger
a = 'Health_Full_600'
csv_logger = CSVLogger('./log'+a+'.csv', append=True, separator=',')
history = model.fit(train_dataset, epochs=30, callbacks=[csv_logger],validation_data= validation_dataset) 

#Save weights
weight_in_path ='./weights/'
model.save(weight_in_path + a +'.h5')

In [0]:
#Evaluation
model.evaluate(test_dataset)

12500/12500 [==============================] - 17s 1ms/step - loss: 0.9697 - acc: 0.6031


[0.9697329196381569, 0.603115]

In [0]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()